In [2]:
from process import event_preprocess, argument_preprocess,load_event, load_argument, Schema2Label
from src.seqlabel_utils import pos2bio
from src.preprocess.str_utils import get_useless_chars
from process import text_preprocess, gen_pos
from src.dataset.converter import single_text, double_text

In [8]:
train_e = load_event('./trainsample/duee_train.json')
valid_e = load_event('./trainsample/duee_dev.json')
test_e = load_event('./trainsample/duee_test2.json')
schema = Schema2Label('./trainsample/duee_event_schema.json')
train_e.shape, valid_e.shape, test_e.shape

((11908, 3), (1492, 3), (34904, 2))

## pipeline: EventType + Arguments

### 方案1：Trigger抽取序列标注问题

In [9]:
useless_chars = get_useless_chars(train_e['text'].values.tolist() +\
                                  valid_e['text'].values.tolist() +\
                                  test_e['text'].values.tolist())

In [23]:
train_e = event_preprocess(train_e, useless_chars)
valid_e = event_preprocess(valid_e, useless_chars)
test_e = event_preprocess(test_e, useless_chars)
valid_e.to_csv('./trainsample/valid_event.csv')
train_e.to_csv('./trainsample/train_event.csv')
test_e.to_csv('./trainsample/test_event.csv')

In [11]:
from src.dataset.converter import single_text
single_text(train_e['id'].values, train_e['clean_text'].values, train_e['event_bio_label'].values,
            './trainsample','train_event_bio')
single_text(valid_e['id'].values, valid_e['clean_text'].values, valid_e['event_bio_label'].values, 
            './trainsample','valid_event_bio')
single_text(test_e['id'].values, test_e['clean_text'].values, None ,'./trainsample','test_event_bio')

### 事件抽取方案2: 多标签分类问题

In [21]:
single_text(train_e['id'].values, train_e['clean_text'].values, train_e['event_label'].values,
            './trainsample','train_event')
single_text(valid_e['id'].values, valid_e['clean_text'].values, valid_e['event_label'].values, 
            './trainsample','valid_event')
single_text(test_e['id'].values, test_e['clean_text'].values, None, './trainsample','test_event')

### 事件抽取方案3：多标签分类 + 事件类别作为文本输入

In [44]:
enhancement = ''
pos = 1 
pos_list = [pos]

for i,j in enumerate(schema.event_label):
    event = j.replace('/','').replace('-','')
    enhancement+='[E{}]'.format(i) + event 
    pos+=len(event)+1
    pos_list.append(pos)
    
','.join([str(i) for i in pos_list])

'1,10,17,24,31,38,45,52,59,66,73,80,87,94,101,106,111,116,121,126,133,140,147,154,161,168,174,179,184,189,194,199,204,209,214,219,224,229,236,243,250,257,264,271,278,285,292,299,306,313,320,328,335,342,349,357,364,371,378,385,392,399,406,413,420,427'

In [35]:
double_text(train_e['id'].values, [enhancement] * train_e.shape[0], 
            train_e['clean_text'].values, train_e['event_label'].values,
            './trainsample','train_event_slot')
double_text(valid_e['id'].values, [enhancement] * valid_e.shape[0],
            valid_e['clean_text'].values, valid_e['event_label'].values, 
            './trainsample','valid_event_slot')
double_text(test_e['id'].values, [enhancement] * test_e.shape[0], 
            test_e['clean_text'].values, None, './trainsample','test_event_slot')

### Argument抽取

In [18]:
from process import process_stage1_test

In [22]:
train_a = argument_preprocess(train_a, useless_chars)
valid_a = argument_preprocess(valid_a, useless_chars)
test_a = argument_preprocess(test_a, useless_chars)

In [23]:
train_a.to_csv('./trainsample/train_argument.csv')
valid_a.to_csv('./trainsample/valid_argument.csv')
test_a.to_csv('./trainsample/test_argument.csv')

## MTL: EventType + Arugments

## Flatten: EventType * Arguments
